## Neural networks with PyTorch
Deep learning networks tend to be massive with dozens or hundreds of layers, that's where the term "deep" comes from. You can build one of these deep networks using only weight matrices as we did in the previous notebook, but in general it's very cumbersome and difficult to implement. PyTorch has a nice module nn that provides a nice way to efficiently build large neural networks.


In [4]:
# Import necessary packages

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
!pip install grp
import numpy as np
import torch
import helper
import matplotlib.pyplot as plt

ERROR: Could not find a version that satisfies the requirement grp (from versions: none)
ERROR: No matching distribution found for grp


ModuleNotFoundError: No module named 'grp'



Now we're going to build a larger network that can solve a (formerly) difficult problem, identifying text in an image. Here we'll use the MNIST dataset which consists of greyscale handwritten digits. Each image is 28x28 pixels, you can see a sample below

(picture handwritten digits)



Our goal is to build a neural network that can take one of these images and predict the digit in the image.

First up, we need to get our dataset. This is provided through the torchvision package. The code below will download the MNIST dataset, then create training and test datasets for us. Don't worry too much about the details here, you'll learn more about this later.


In [ ]:

### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)


We have the training data loaded into trainloader and we make that an iterator with iter(trainloader). Later, we'll use this to loop through the dataset for training, like

for image, label in trainloader:
    ## do things with images and labels
You'll notice I created the trainloader with a batch size of 64, and shuffle=True. The batch size is the number of images we get in one iteration from the data loader and pass through our network, often called a batch. And shuffle=True tells it to shuffle the dataset every time we start going through the data loader again. But here I'm just grabbing the first batch so we can check out the data. We can see below that images is just a tensor with size (64, 1, 28, 28). So, 64 images per batch, 1 color channel, and 28x28 images.


In [ ]:

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

In [ ]:
#This is what one of the images looks like.


plt.imshow(images[8].numpy().squeeze(), cmap='Greys_r');
labels[8]

First, let's try to build a simple network for this dataset using weight matrices and matrix multiplications. Then, we'll see how to do it using PyTorch's nn module which provides a much more convenient and powerful method for defining network architectures.

The networks you've seen so far are called fully-connected or dense networks. Each unit in one layer is connected to each unit in the next layer. In fully-connected networks, the input to each layer must be a one-dimensional vector (which can be stacked into a 2D tensor as a batch of multiple examples). However, our images are 28x28 2D tensors, so we need to convert them into 1D vectors. Thinking about sizes, we need to convert the batch of images with shape (64, 1, 28, 28) to a have a shape of (64, 784), 784 is 28 times 28. This is typically called flattening, we flattened the 2D images into 1D vectors.

Previously you built a network with one output unit. Here we need 10 output units, one for each digit. We want our network to predict the digit shown in an image, so what we'll do is calculate probabilities that the image is of any one digit or class. This ends up being a discrete probability distribution over the classes (digits) that tells us the most likely class for the image. That means we need 10 output units for the 10 classes (digits). We'll see how to convert the network output into a probability distribution next.

Exercise: Flatten the batch of images images. Then build a multi-layer network with 784 input units, 256 hidden units, and 10 output units using random tensors for the weights and biases. For now, use a sigmoid activation for the hidden layer. Leave the output layer without an activation, we'll add one that gives us a probability distribution next.


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#!pip install grp
import numpy as np
import torch
#import helper
import matplotlib.pyplot as plt

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)    

dataiter = iter(trainloader)
images, labels = dataiter.next()
#features = torch.flatten(images,start_dim=1)
#W1,W2,B1,B2 = multi_Layer_NW(features.shape[1],256,10)
#input (64,1,28,28) 64 bilder i en batch, 1 channel grey, pixel*pixel
#flatten till (64,784)

n_input = 784    #28*28 pixlar
n_hidden = 256  #?
n_out = 10       # 10 olika siffror

inputs = images.view(images.shape[0],-1)  #go from 2D to 1D
inputs = images.view(images.shape[0],784) #equivalent, manually specified
w1 = torch.randn(n_input,n_hidden)
w2 = torch.randn(n_hidden, n_out)
b1 = torch.randn(n_hidden) # size must match dimensions
b2 = torch.randn(n_out) #match dim w2

h = activation(torch.mm(inputs,w1)+b1)
out = activation(torch.mm(h,w2)+b2)

#out1 = calc_output(features,W1,W2,B1,B2) # output of your network, should have shape (64,10)

print(inputs.shape, out.shape, out1.shape)

In [ ]:



## Your solution   

## Activation function
def activation(x):
    """ Sigmoid activation function 
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

### Neural network
def multi_Layer_NW(inputUnits, hiddenUnits, outputUnits):
    torch.manual_seed(7) # Set the random seed so things are predictable

    # Define the size of each layer in our network
    n_input = inputUnits     # Number of input units, must match number of input features
    n_hidden = hiddenUnits   # Number of hidden units 
    n_output = outputUnits   # Number of output units

    # Weights for inputs to hidden layer
    W1 = torch.randn(n_input, n_hidden)
    # Weights for hidden layer to output layer
    W2 = torch.randn(n_hidden, n_output)

    # and bias terms for hidden and output layers
    B1 = torch.randn((1, n_hidden))
    B2 = torch.randn((1, n_output))

    return W1,W2,B1,B2

def calc_output(features,W1,W2,B1,B2):
    h = activation(torch.matmul(features,W1).add_(B1))
    output = activation(torch.matmul(h,W2).add_(B2))
    return output

# Features are flattened batch input
#features = torch.flatten(images,start_dim=1)
#W1,W2,B1,B2 = multi_Layer_NW(features.shape[1],256,10)




    






inputUnits = 784       
hiddenUnits = 256
outputUnits = 10


Now we have 10 outputs for our network. We want to pass in an image to our network and get out a probability distribution over the classes that tells us the likely class(es) the image belongs to. Something that looks like this:

Here we see that the probability for each class is roughly the same. This is representing an untrained network, it hasn't seen any data yet so it just returns a uniform distribution with equal probabilities for each class.

To calculate this probability distribution, we often use the softmax function. Mathematically this looks like

$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$
What this does is squish each input $x_i$ between 0 and 1 and normalizes the values to give you a proper probability distribution where the probabilites sum up to one.

Exercise: Implement a function softmax that performs the softmax calculation and returns probability distributions for each example in the batch. Note that you'll need to pay attention to the shapes when doing this. If you have a tensor a with shape (64, 10) and a tensor b with shape (64,), doing a/b will give you an error because PyTorch will try to do the division across the columns (called broadcasting) but you'll get a size mismatch. The way to think about this is for each of the 64 examples, you only want to divide by one value, the sum in the denominator. So you need b to have a shape of (64, 1). This way PyTorch will divide the 10 values in each row of a by the one value in each row of b. Pay attention to how you take the sum as well. You'll need to define the dim keyword in torch.sum. Setting dim=0 takes the sum across the rows while dim=1 takes the sum across the columns.

Det är 64 rader med olika bilder, vill ha sannolikhet 1 totalt för varje bild. Så varje bild ska tillhöra någon siffra. output prop måste vara lika många element som det är bilder in i  funktionen. Alltså ut (64,1)
I SoftMax vill vi ha exp(x) / summan. x=torch(64, 10) dim0==rad, dim1==column, vi vill behålla 64 och summera över de 10 möjligheterna. Alltså välja dim=1.
Då är tellare: (64,10), nämnare vector (64), måste reshapa nämnaren så den åter är en tensor


In [ ]:
def softmax(x):
    ## TODO: Implement the softmax function here
    #calc for each in input-vektor
    #sum it up and normalize
    print('x', x.shape)
    
    return torch.exp(x) / torch.sum(torch.exp(x), dim=1).view(-1,1)


n_input = 784    #28*28 pixlar
n_hidden = 256  #?
n_out = 10       # 10 olika siffror

inputs = images.view(images.shape[0],-1)  #go from 2D to 1D
#inputs = images.view(images.shape[0],784) #equivalent, manually specified
w1 = torch.randn(n_input,n_hidden)
w2 = torch.randn(n_hidden, n_out)
b1 = torch.randn(n_hidden) # size must match dimensions
b2 = torch.randn(n_out) #match dim w2

h = activation(torch.mm(inputs,w1)+b1)
out = activation(torch.mm(h,w2)+b2)

#out = calc_output(features,W1,W2,B1,B2) # output of your network, should have shape (64,10)

print('inputs, outputs',inputs.shape, out.shape)



out2=out.detach()
# Here, out should be the output of the network in the previous excercise with shape (64,10)
probabilities = softmax(out2)

# Does it have the right shape? Should be (64, 10)
print('probabilities',probabilities.shape)
# Does it sum to 1?
print('does it sum to 1?',probabilities.sum(dim=1))

In [ ]:
def softmax(x):
    ## TODO: Implement the softmax function here
    #calc for each in input-vektor
    #sum it up and normalize
    
    return torch.exp(x) / torch.sum(torch.exp(x), dim=1).view(-1,1)


n_input = 784    #28*28 pixlar
n_hidden = 256  #?
n_out = 10       # 10 olika siffror

inputs = images.view(images.shape[0],-1)  #go from 2D to 1D
#inputs = images.view(images.shape[0],784) #equivalent, manually specified
w1 = torch.randn(n_input,n_hidden)
w2 = torch.randn(n_hidden, n_out)
b1 = torch.randn(n_hidden) # size must match dimensions
b2 = torch.randn(n_out) #match dim w2

h = activation(torch.mm(inputs,w1)+b1)
out = activation(torch.mm(h,w2)+b2)

#out = calc_output(features,W1,W2,B1,B2) # output of your network, should have shape (64,10)

print('inputs, outputs',inputs.shape, out.shape)



out2=out.detach()
# Here, out should be the output of the network in the previous excercise with shape (64,10)
probabilities = softmax(out2)

# Does it have the right shape? Should be (64, 10)
print('probabilities',probabilities.shape)
# Does it sum to 1?
print('does it sum to 1?',probabilities.sum(dim=1))

## Building networks with PyTorch
PyTorch provides a module nn that makes building networks much simpler. Here I'll show you how to build the same one as above with 784 inputs, 256 hidden units, 10 output units and a softmax output.


In [ ]:
from torch import nn
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)
        
        # Define sigmoid activation and softmax output 
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x


Let's go through this bit by bit.

class Network(nn.Module):
Here we're inheriting from nn.Module. Combined with super().__init__() this creates a class that tracks the architecture and provides a lot of useful methods and attributes. It is mandatory to inherit from nn.Module when you're creating a class for your network. The name of the class itself can be anything.

self.hidden = nn.Linear(784, 256)
This line creates a module for a linear transformation, $x\mathbf{W} + b$, with 784 inputs and 256 outputs and assigns it to self.hidden. The module automatically creates the weight and bias tensors which we'll use in the forward method. You can access the weight and bias tensors once the network (net) is created with net.hidden.weight and net.hidden.bias.

self.output = nn.Linear(256, 10)
Similarly, this creates another linear transformation with 256 inputs and 10 outputs.

self.sigmoid = nn.Sigmoid()
self.softmax = nn.Softmax(dim=1)
Here I defined operations for the sigmoid activation and softmax output. Setting dim=1 in nn.Softmax(dim=1) calculates softmax across the columns.

def forward(self, x):
PyTorch networks created with nn.Module must have a forward method defined. It takes in a tensor x and passes it through the operations you defined in the __init__ method.

x = self.hidden(x)
x = self.sigmoid(x)
x = self.output(x)
x = self.softmax(x)
Here the input tensor x is passed through each operation and reassigned to x. We can see that the input tensor goes through the hidden layer, then a sigmoid function, then the output layer, and finally the softmax function. It doesn't matter what you name the variables here, as long as the inputs and outputs of the operations match the network architecture you want to build. The order in which you define things in the __init__ method doesn't matter, but you'll need to sequence the operations correctly in the forward method.

Now we can create a Network object.


In [ ]:
# Create the network and look at its text representation
model = Network()
model



You can define the network somewhat more concisely and clearly using the torch.nn.functional module. This is the most common way you'll see networks defined as many operations are simple element-wise functions. We normally import this module as F, import torch.nn.functional as F.


In [ ]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(784, 256)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        # Hidden layer with sigmoid activation
        x = F.sigmoid(self.hidden(x))
        # Output layer with softmax activation
        x = F.softmax(self.output(x), dim=1)
        
        return x


## Activation functions
So far we've only been looking at the sigmoid activation function, but in general any function can be used as an activation function. The only requirement is that for a network to approximate a non-linear function, the activation functions must be non-linear. Here are a few more examples of common activation functions: Tanh (hyperbolic tangent), and ReLU (rectified linear unit).

(figures)

In practice, the ReLU function is used almost exclusively as the activation function for hidden layers.
Nätverks brukar lära sig snabbare med ReLU än med sigmoid eller tanh

### Your Turn to Build a Network


Exercise: Create a network with 784 input units, a hidden layer with 128 units and a ReLU activation, then a hidden layer with 64 units and a ReLU activation, and finally an output layer with a softmax activation as shown above. You can use a ReLU activation with the nn.ReLU module or F.relu function.

It's good practice to name your layers by their type of network, for instance 'fc' to represent a fully-connected layer. As you code your solution, use fc1, fc2, and fc3 as your layer names.


In [ ]:
## Your solution here
#Initializing weights and biases
#The weights and such are automatically initialized for you, but it's possible to customize how they are initialized. The weights and biases are tensors attached to the layer you defined, you can get them with model.fc1.weight for instance.

import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        # Output layer, 10 units - one for each digit
        self.fc3 = nn.Linear(64, 10)
        
        
    def forward(self, x):
        # Hidden layer with ReLU activation
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        # Output layer with softmax activation
        x = F.softmax(x, dim=1)
        
        return x
model = Network()
model


In [ ]:

print(model.fc1.weight)
print(model.fc1.bias)

In [ ]:
#For custom initialization, we want to modify these tensors in place. These are actually autograd Variables, 
#so we need to get back the actual tensors with model.fc1.weight.data. Once we have the tensors, 
#we can fill them with zeros (for biases) or random normal values.

# Set biases to all zeros
model.fc1.bias.data.fill_(0)

In [ ]:
# sample from random normal with standard dev = 0.01
model.fc1.weight.data.normal_(std=0.01)


## Forward pass
Now that we have a network, let's see what happens when we pass in an image.

In [ ]:
import helper
# Grab some data 
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Resize images into a 1D vector, new shape is (batch size, color channels, image pixels) 
images.resize_(64, 1, 784)
# or images.resize_(images.shape[0], 1, 784) to automatically get batch size

# Forward pass through the network
img_idx = 0
ps = model.forward(images[img_idx,:])

img = images[img_idx]
helper.view_classify(img.view(1, 28, 28), ps)



As you can see above, our network has basically no idea what this digit is. It's because we haven't trained it yet, all the weights are random!

## Using nn.Sequential
PyTorch provides a convenient way to build networks like this where a tensor is passed sequentially through operations, nn.Sequential (documentation). Using this to build the equivalent network:


In [ ]:
# Hyperparameters for our network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

# Forward pass through the network and display output
images, labels = next(iter(trainloader))
images.resize_(images.shape[0], 1, 784)
ps = model.forward(images[0,:])
helper.view_classify(images[0].view(1, 28, 28), ps)

In [ ]:
print(model[0])
model[0].weight

Linear(in_features=784, out_features=128, bias=True)
Out[66]:
Parameter containing:
tensor([[-0.0087, -0.0220,  0.0054,  ..., -0.0126,  0.0284, -0.0057],
        [-0.0007,  0.0179, -0.0247,  ...,  0.0131, -0.0338, -0.0108],
        [ 0.0012,  0.0208, -0.0094,  ...,  0.0140,  0.0052,  0.0094],
        ...,
        [-0.0183,  0.0111,  0.0117,  ...,  0.0324,  0.0155, -0.0284],
        [ 0.0046, -0.0355,  0.0147,  ...,  0.0157,  0.0112, -0.0042],
        [ 0.0255, -0.0254, -0.0215,  ...,  0.0253, -0.0181,  0.0345]],
       requires_grad=True)
You can also pass in an OrderedDict to name the individual layers and operations, instead of using incremental integers. Note that dictionary keys must be unique, so each operation must have a different name.



Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): Softmax(dim=1)
)

Here our model is the same as before: 784 input units, a hidden layer with 128 units, ReLU activation, 64 unit hidden layer, another ReLU, then the output layer with 10 units, and the softmax output.

The operations are available by passing in the appropriate index. For example, if you want to get first Linear operation and look at the weights, you'd use model[0].


In [ ]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('softmax', nn.Softmax(dim=1))]))
model





Sequential(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (output): Linear(in_features=64, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)
Now you can access layers either by integer or the name



In [ ]:
print(model[0])
print(model.fc1)


Linear(in_features=784, out_features=128, bias=True)
Linear(in_features=784, out_features=128, bias=True)
In the next notebook, we'll see how we can train a neural network to accuractly predict the numbers appearing in the MNIST images.